# Zadanie: Implementacja Sieci Semantycznej dla Systemu Bibliotecznego

## Cel zadania

Celem zadania jest stworzenie sieci semantycznej reprezentującej wiedzę o książkach, autorach, gatunkach literackich i powiązaniach między nimi w systemie bibliotecznym. System ma umożliwiać wyszukiwanie i rekomendowanie książek na podstawie różnych kryteriów i relacji.

## Opis systemu

System powinien przechowywać i przetwarzać informacje o:
1. Książkach (tytuł, rok wydania, język)
2. Autorach (imię, nazwisko, okres twórczości)
3. Gatunkach literackich (hierarchia gatunków)
4. Motywach i tematach
5. Powiązaniach między książkami (kontynuacje, inspiracje)

## Wymagania funkcjonalne

1. Implementacja klasy `LibrarySemanticNetwork`
2. Metody do zarządzania węzłami i relacjami:
   - Dodawanie/usuwanie książek, autorów, gatunków
   - Tworzenie różnych typów relacji
   - Modyfikacja istniejących powiązań
3. Metody wyszukiwania i rekomendacji:
   - Znajdowanie powiązanych książek
   - Wyszukiwanie po motywach
   - Rekomendacje na podstawie podobieństw
4. System wnioskowania:
   - Dziedziczenie cech gatunków
   - Wykrywanie pośrednich powiązań
   - Analiza ścieżek między węzłami

## Struktura implementacji

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Set, Optional
from datetime import datetime
import networkx as nx
import matplotlib.pyplot as plt

@dataclass
class Book:
    title: str
    year: int
    language: str
    isbn: str

@dataclass
class Author:
    name: str
    birth_year: int
    death_year: Optional[int]
    nationality: str

@dataclass
class Genre:
    name: str
    parent_genre: Optional['Genre']
    characteristics: List[str]

class LibrarySemanticNetwork:
    def __init__(self):
        self.books: Dict[str, Book] = {}
        self.authors: Dict[str, Author] = {}
        self.genres: Dict[str, Genre] = {}
        self.relations: Dict[str, Dict[str, Set[tuple]]] = {}
        self.motifs: Dict[str, Set[str]] = {}
        self.graph = nx.DiGraph()

    def add_book(self, book: Book):
        """Dodaje książkę do sieci"""
        if book.isbn not in self.books:
            self.books[book.isbn] = book
            self.graph.add_node(book.isbn, type='book', data=book)
            self.relations[book.isbn] = {}

    def add_author(self, author: Author):
        """Dodaje autora do sieci"""
        if author.name not in self.authors:
            self.authors[author.name] = author
            self.graph.add_node(author.name, type='author', data=author)
            self.relations[author.name] = {}

    def add_genre(self, genre: Genre):
        """Dodaje gatunek do sieci"""
        if genre.name not in self.genres:
            self.genres[genre.name] = genre
            self.graph.add_node(genre.name, type='genre', data=genre)
            self.relations[genre.name] = {}
            if genre.parent_genre:
                self.add_relation(genre.name, 'is_subgenre_of', genre.parent_genre.name)

    def add_relation(self, source: str, relation_type: str, target: str):
        """Dodaje relację między węzłami"""
        if source in self.relations:
            if relation_type not in self.relations[source]:
                self.relations[source][relation_type] = set()
            self.relations[source][relation_type].add(target)
            self.graph.add_edge(source, target, relation=relation_type)

    def add_motif(self, motif: str, book_isbn: str):
        """Dodaje motyw do książki"""
        if motif not in self.motifs:
            self.motifs[motif] = set()
        self.motifs[motif].add(book_isbn)
        self.graph.add_node(motif, type='motif')
        self.graph.add_edge(book_isbn, motif, relation='has_motif')

    def find_related_books(self, book_isbn: str, max_distance: int = 2) -> List[str]:
        """Znajduje książki powiązane z daną książką"""
        related = []
        if book_isbn in self.books:
            for node in nx.single_source_shortest_path_length(self.graph, book_isbn, cutoff=max_distance):
                if node in self.books and node != book_isbn:
                    related.append(node)
        return related

    def find_books_by_motif(self, motif: str) -> List[str]:
        """Znajduje książki zawierające dany motyw"""
        return list(self.motifs.get(motif, set()))

    def recommend_books(self, book_isbn: str, n: int = 5) -> List[str]:
        """Rekomenduje książki na podstawie podobieństw"""
        if book_isbn not in self.books:
            return []

        # Implementacja systemu rekomendacji
        # TODO: Zaimplementuj algorytm rekomendacji
        pass

    def find_path_between_books(self, book1_isbn: str, book2_isbn: str) -> List[str]:
        """Znajduje ścieżkę między dwiema książkami"""
        try:
            return nx.shortest_path(self.graph, book1_isbn, book2_isbn)
        except nx.NetworkXNoPath:
            return []

    def visualize(self):
        """Wizualizuje sieć semantyczną"""
        plt.figure(figsize=(12, 8))
        pos = nx.spring_layout(self.graph)
        
        # Rysuj węzły różnych typów
        node_colors = {
            'book': 'skyblue',
            'author': 'lightgreen',
            'genre': 'salmon',
            'motif': 'yellow'
        }
        
        for type_name, color in node_colors.items():
            nodes = [n for n, d in self.graph.nodes(data=True) if d.get('type') == type_name]
            nx.draw_networkx_nodes(self.graph, pos, nodelist=nodes, node_color=color, node_size=500)
        
        nx.draw_networkx_edges(self.graph, pos, arrows=True)
        nx.draw_networkx_labels(self.graph, pos)
        
        plt.title('Library Semantic Network')
        plt.axis('off')
        plt.show()

## Przykład użycia

In [ ]:
# Tworzenie sieci
network = LibrarySemanticNetwork()

# Dodawanie gatunków
fantasy = Genre('Fantasy', None, ['magic', 'adventure'])
urban_fantasy = Genre('Urban Fantasy', fantasy, ['modern setting', 'magic'])
network.add_genre(fantasy)
network.add_genre(urban_fantasy)

# Dodawanie autorów
author1 = Author('J.R.R. Tolkien', 1892, 1973, 'British')
network.add_author(author1)

# Dodawanie książek
book1 = Book('The Lord of the Rings', 1954, 'English', '978-0618640157')
network.add_book(book1)

# Dodawanie relacji
network.add_relation(book1.isbn, 'written_by', author1.name)
network.add_relation(book1.isbn, 'belongs_to_genre', fantasy.name)

# Dodawanie motywów
network.add_motif('quest', book1.isbn)
network.add_motif('magic_ring', book1.isbn)

# Wizualizacja sieci
network.visualize()

# Przykładowe wyszukiwania
related_books = network.find_related_books(book1.isbn)
books_with_quest = network.find_books_by_motif('quest')

print("Powiązane książki:", related_books)
print("Książki z motywem quest:", books_with_quest)

## Zadania do wykonania

1. Uzupełnij implementację metody `recommend_books`:
   - Wykorzystaj podobieństwo gatunków
   - Uwzględnij wspólne motywy
   - Weź pod uwagę powiązania między autorami

2. Dodaj metody do analizy sieci:
   - Znajdowanie wszystkich książek w danym gatunku i jego podgatunkach
   - Analiza podobieństwa między autorami
   - Wykrywanie grup tematycznych

3. Rozszerz system o nowe funkcjonalności:
   - Obsługa serii książek
   - Śledzenie wpływów literackich
   - Analiza czasowa (trendy w literaturze)

4. Zaimplementuj zaawansowane metody wyszukiwania:
   - Wyszukiwanie rozmyte (fuzzy search)
   - Wyszukiwanie kontekstowe
   - Rankingi podobieństwa

5. Dodaj mechanizmy walidacji i spójności danych:
   - Sprawdzanie poprawności relacji
   - Wykrywanie duplikatów
   - Weryfikacja spójności hierarchii gatunków